In [ ]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import math
import matplotlib.pyplot as plt

In [ ]:
def get_stock_df():
    print('Enter the stock you want to look at (stock symbol): ')
    ticker = input()
    print('Enter the start time (in the form "year-month-date" e.g. 2017-10-01): ')
    start_date = input()
    start_list = start_date.split('-')
    start = datetime.datetime(int(start_list[0]),int(start_list[1]),int(start_list[2]))
    print('Enter the end time (in the form "year-month-date" e.g. 2017-10-01): ')
    end_date = input()
    end_list = end_date.split('-')
    end = datetime.datetime(int(end_list[0]),int(end_list[1]),int(end_list[2]))
    df = yf.download(ticker,start = start,end = end)
    return df

In [ ]:
def get_simple_moving_average(df, window):
    df['Normal Return'] = df['Adj Close'].pct_change()
    df[str(window) + ' SMA'] = df['Adj Close'].rolling(window).mean()
    df['Signal'] = np.where(df[str(window) + 'day MA'] > df['Adj Close'],1,0)
    df['Log Return'] = df['Normal Return'].apply(lambda x:math.log(x+1))
    df['Strat Daily Return'] = np.where(df['Signal'] == 1, -df['Log Return'],0)
    df['Cumulative Sum'] = df['Strat Daily Return'].cumsum()
    plt.plot(df.reset_index()['Date'],df['Adj Close'], label = 'Close Price')
    plt.plot(df.reset_index()['Date'],df[str(window) + ' SMA'], label = str(window) + ' SMA')
    plt.legend()
    plt.title('Cumulative Return in %')
    plt.show()
    if df['Cumulative Sum'][-1] >= 0:
        print('Recomendation:')
        print('The simple moving average startegy of window ' + str(window) + ' yields a positive cumulative return. Hence, the strategy could be taken into consideration.')
    else:
        print('Recommendation:')
        print('The simple moving average startegy of window ' + str(window) + ' yields a negative cumulative return. Hence, the strategy should not be taken into consideration.')
    
    return df,df['Cumulative Sum'][-1]

In [ ]:
def get_exponential_moving_average(df,window):
    df[str(window) + ' EMA'] = pd.Series.ewm(df['Adj Close'], span=window).mean()
    df['Signal'] = np.where(df[str(window) + ' EMA'] > df['Adj Close'],1,0)
    df['Normal Return'] = df['Adj Close'].pct_change()
    df['Log Return'] = df['Normal Return'].apply(lambda x:math.log(x+1))
    df['Strat Daily Return'] = np.where(df['Signal'] == 1, -df['Log Return'],0)
    df['Cumulative Sum'] = df['Strat Daily Return'].cumsum()
    plt.plot(df.reset_index()['Date'],df['Adj Close'], label = 'Close Price')
    plt.plot(df.reset_index()['Date'],df[str(window) + ' EMA'], label = str(window) + ' EMA')
    plt.legend()
    plt.title('Cumulative Return in %')
    plt.show()
    if df['Cumulative Sum'][-1] >= 0:
        print('Recomendation:')
        print('The exponential moving average startegy of window ' + str(window) + ' yields a positive cumulative return. Hence, the strategy could be taken into consideration.')
    else:
        print('Recommendation:')
        print('The exponential moving average startegy of window ' + str(window) + ' yields a negative cumulative return. Hence, the strategy should not be taken into consideration.')
    return df,df['Cumulative Sum'][-1]